### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_stones/GSE11138'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True  # microarray implies gene expression data is available

# Step 2: Variable Availability and Data Type Conversion

# Identify the key in the sample characteristics dictionary where the variable is recorded
trait_row = 5  # 'comorbidities' contains 'Stone Kidney'
age_row = 2    # 'age (years)' contains age information
gender_row = 1 # 'sex' contains gender information

# Define conversion functions

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        if "Stone Kidney" in value:
            return 1
        elif "-" in value or "Cholecystectomy" in value:  # assuming lack of explicit mention of stones equates to no kidney stones
            return 0
        else:
            return None
    except:
        return None 

def convert_age(value):
    try:
        return int(value.split(': ')[-1])  # Extract integer age value
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(': ')[-1].strip().lower()
        if gender == 'm':
            return 1
        elif gender == 'f':
            return 0
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE11138', './preprocessed/Kidney_stones/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_stones', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_stones/trait_data/GSE11138.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step
print(gene_data.index[:20])


import re

# Placeholder for the list of valid human gene symbols
# Note: The actual list should be comprehensive and can be fetched from a gene symbol database like HGNC.
human_gene_symbols = set(["A2M", "P53", "BRCA1", "TP53"])  # Example, needs to be comprehensive.

def requires_gene_mapping(headers):
    if not headers:  # Check if headers list is empty or not defined
        return True
    for header in headers:
        if not re.match(r"^[a-zA-Z0-9_\-]+$", header) or header not in human_gene_symbols:
            return True
    return False


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# Ensure gene_data is correctly defined by reloading it
gene_data = get_genetic_data(matrix_file)

# Re-preview gene_annotation to ensure correct keys are used
print("Gene annotation columns:", gene_annotation.columns)

# From previous preview we can see the keys
identifier_key = 'ID'
gene_symbol_key = 'Gene_Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Preview the mapping to ensure correctness
print("Gene mapping preview:")
print(preview_df(gene_mapping_df))

# 4. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# 5. Print the first 20 row IDs of the mapped gene data to check correctness
print(gene_data.index[:20])


### Step 7: Data Normalization and Merging

In [ ]:
# Ensure `gene_data` obtained correctly (This assumes `STEP6` fixed and returned `gene_data`)
try:
    if gene_data is None:
        raise ValueError("gene_data is not defined. Ensure STEP6 is executed correctly.")
except NameError:
    raise ValueError("gene_data is not defined. Ensure STEP6 is executed correctly.")

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_stones/gene_data/GSE11138.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait 'Kidney_stones' and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_stones')

# 4. Save the cohort information.
save_cohort_info('GSE11138', './preprocessed/Kidney_stones/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_stones/GSE11138.csv'
    unbiased_merged_data.to_csv(csv_path)
